# SMTP Diagnostics (ENV-first) — OurFinanceTracker

This notebook assumes it lives **inside your OurFinanceTracker project**.  
It will:
- Load **environment variables** from your runtime (Render/Replit/local) and from a `.env` file in the project root if present.
- Resolve the SMTP host (DNS).
- Probe **STARTTLS (587)** and **SSL (465)** using values from the environment (first) or Django settings (fallback).
- Optionally send a message using Django's e-mail backend.

> Everything is written in British English. No overrides are applied unless a value is missing in the environment.

In [ ]:
# --- Configuration you might tweak ---
TEST_TO = "you@example.com"   # Set a real e-mail to receive the probe
SEND_EMAIL = True             # Set False to perform connection-only checks

# If DJANGO_SETTINGS_MODULE is not defined in the environment, we will fall back to this:
FALLBACK_SETTINGS_MODULE = "ourfinancetracker_site.settings"

In [ ]:
import os, sys, ssl, smtplib, socket, traceback
from email.message import EmailMessage
from pathlib import Path
from datetime import datetime

def env_bool(v, default=False):
    if v is None:
        return default
    return str(v).strip().lower() in ("1","true","yes","on")

def mask_secret(s, keep=3):
    if not s:
        return ""
    s = str(s)
    if len(s) <= keep:
        return "*" * len(s)
    return "*" * (len(s)-keep) + s[-keep:]

def find_project_root(start: Path) -> Path:
    # Walk upwards until we find 'manage.py' or a '.git' or hit the filesystem root
    p = start.resolve()
    while True:
        if (p / "manage.py").exists() or (p / ".git").exists():
            return p
        if p.parent == p:
            return start.resolve()
        p = p.parent

def load_dotenv_loose(dotenv_path: Path):
    # Lightweight .env parser that does not require python-dotenv.
    if not dotenv_path.exists():
        return False
    try:
        for line in dotenv_path.read_text(encoding="utf-8").splitlines():
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            if "=" not in line:
                continue
            k, v = line.split("=", 1)
            k = k.strip()
            v = v.strip().strip("'").strip('"')
            # Do not overwrite existing environment variables
            if k not in os.environ:
                os.environ[k] = v
        return True
    except Exception as e:
        print("Warning: failed to parse .env loosely:", e)
        return False

def current_context_name():
    if os.getenv("RENDER") or os.getenv("RENDER_SERVICE_NAME"):
        return "Render (production-like)"
    if os.getenv("REPL_ID") or os.getenv("REPLIT_DB_URL"):
        return "Replit (development)"
    return "Local (unknown host)"

def resolve_host(host):
    print(f"Resolving host: {host}")
    try:
        info = socket.getaddrinfo(host, None)
        addrs = sorted({item[4][0] for item in info})
        for i, addr in enumerate(addrs, 1):
            print(f"  [{i}] {addr}")
        if not addrs:
            print("  No addresses found.")
    except Exception as e:
        print("  DNS resolution failed:", e)

def build_message(from_email, to_email, subject, body):
    msg = EmailMessage()
    msg["From"] = from_email
    msg["To"] = to_email
    msg["Subject"] = subject
    msg.set_content(body)
    return msg

def try_starttls(host, port, user, password, from_email, to_email, timeout=10, perform_send=True):
    print(f"\n--- STARTTLS probe: {host}:{port} ---")
    try:
        with smtplib.SMTP(host, int(port), timeout=int(timeout)) as s:
            s.ehlo()
            s.starttls(context=ssl.create_default_context())
            s.ehlo()
            if user:
                s.login(user, password or "")
            print("Connection + STARTTLS + (optional) login: OK")
            if perform_send:
                msg = build_message(from_email, to_email,
                    f"SMTP STARTTLS probe @ {datetime.utcnow().isoformat()}Z",
                    "Hello! This is a STARTTLS SMTP diagnostic message from OurFinanceTracker notebook."
                )
                s.send_message(msg)
                print("Send: OK")
        print("STARTTLS probe finished successfully.")
    except Exception as e:
        print("STARTTLS probe FAILED.")
        print("Type:", type(e).__name__)
        print("Message:", str(e))
        print("Traceback:")
        traceback.print_exc()

def try_ssl(host, port, user, password, from_email, to_email, timeout=10, perform_send=True):
    print(f"\n--- SSL (465) probe: {host}:{port} ---")
    try:
        ctx = ssl.create_default_context()
        with smtplib.SMTP_SSL(host, int(port), context=ctx, timeout=int(timeout)) as s:
            s.ehlo()
            if user:
                s.login(user, password or "")
            print("SSL connection + (optional) login: OK")
            if perform_send:
                msg = build_message(from_email, to_email,
                    f"SMTP SSL(465) probe @ {datetime.utcnow().isoformat()}Z",
                    "Hello! This is an SSL(465) SMTP diagnostic message from OurFinanceTracker notebook."
                )
                s.send_message(msg)
                print("Send: OK")
        print("SSL(465) probe finished successfully.")
    except Exception as e:
        print("SSL(465) probe FAILED.")
        print("Type:", type(e).__name__)
        print("Message:", str(e))
        print("Traceback:")
        traceback.print_exc()

# 1) Locate project root and load .env if present
nb_path = Path.cwd()
proj_root = find_project_root(nb_path)
dotenv_path = proj_root / ".env"
loaded = load_dotenv_loose(dotenv_path)
print(f"Context: {current_context_name()}")
print(f"Notebook cwd: {nb_path}")
print(f"Project root: {proj_root}")
print(f".env loaded: {loaded} ({dotenv_path if loaded else 'not found or failed'})")

# 2) Ensure DJANGO_SETTINGS_MODULE set if not present
if not os.getenv("DJANGO_SETTINGS_MODULE"):
    os.environ["DJANGO_SETTINGS_MODULE"] = os.getenv("DJANGO_SETTINGS_MODULE", FALLBACK_SETTINGS_MODULE)
    print("DJANGO_SETTINGS_MODULE set to:", os.environ["DJANGO_SETTINGS_MODULE"])
else:
    print("DJANGO_SETTINGS_MODULE already present:", os.environ["DJANGO_SETTINGS_MODULE"])

In [ ]:
# 3) Gather configuration from ENV first; fallback to Django settings if missing.
EMAIL_BACKEND = os.getenv("EMAIL_BACKEND")
EMAIL_HOST = os.getenv("EMAIL_HOST")
EMAIL_PORT = os.getenv("EMAIL_PORT")
EMAIL_HOST_USER = os.getenv("EMAIL_HOST_USER")
EMAIL_HOST_PASSWORD = os.getenv("EMAIL_HOST_PASSWORD")
EMAIL_USE_TLS = os.getenv("EMAIL_USE_TLS")
EMAIL_USE_SSL = os.getenv("EMAIL_USE_SSL")
EMAIL_TIMEOUT = os.getenv("EMAIL_TIMEOUT")
DEFAULT_FROM_EMAIL = os.getenv("DEFAULT_FROM_EMAIL")

# Attempt to import Django settings for fallback values
django_loaded = False
try:
    import django
    django.setup()
    from django.conf import settings as djset
    django_loaded = True
    print("Django loaded for fallback settings.")
except Exception as e:
    print("Django not available yet or failed to load:", e)

def fallback(v, default):
    return default if (v is None or v == "") else v

if django_loaded:
    EMAIL_BACKEND      = fallback(EMAIL_BACKEND, getattr(djset, "EMAIL_BACKEND", None))
    EMAIL_HOST         = fallback(EMAIL_HOST, getattr(djset, "EMAIL_HOST", None))
    EMAIL_PORT         = fallback(EMAIL_PORT, str(getattr(djset, "EMAIL_PORT", "")))
    EMAIL_HOST_USER    = fallback(EMAIL_HOST_USER, getattr(djset, "EMAIL_HOST_USER", None))
    EMAIL_HOST_PASSWORD= fallback(EMAIL_HOST_PASSWORD, getattr(djset, "EMAIL_HOST_PASSWORD", None))
    EMAIL_USE_TLS      = fallback(EMAIL_USE_TLS, str(getattr(djset, "EMAIL_USE_TLS", "")))
    EMAIL_USE_SSL      = fallback(EMAIL_USE_SSL, str(getattr(djset, "EMAIL_USE_SSL", "")))
    EMAIL_TIMEOUT      = fallback(EMAIL_TIMEOUT, str(getattr(djset, "EMAIL_TIMEOUT", "")))
    DEFAULT_FROM_EMAIL = fallback(DEFAULT_FROM_EMAIL, getattr(djset, "DEFAULT_FROM_EMAIL", None))

# Final coercions / defaults
EMAIL_PORT = int(EMAIL_PORT or 587)
EMAIL_USE_TLS = env_bool(EMAIL_USE_TLS if EMAIL_USE_TLS is not None else "true", True)
EMAIL_USE_SSL = env_bool(EMAIL_USE_SSL if EMAIL_USE_SSL is not None else "false", False)
EMAIL_TIMEOUT = int(EMAIL_TIMEOUT or 10)

FROM = DEFAULT_FROM_EMAIL or EMAIL_HOST_USER or "no-reply@ourfinancetracker.com"

print("\n=== Effective configuration (ENV-first, passwords masked) ===")
print("EMAIL_BACKEND      =", EMAIL_BACKEND)
print("EMAIL_HOST         =", EMAIL_HOST)
print("EMAIL_PORT         =", EMAIL_PORT)
print("EMAIL_HOST_USER    =", EMAIL_HOST_USER)
print("EMAIL_HOST_PASSWORD=", mask_secret(EMAIL_HOST_PASSWORD))
print("EMAIL_USE_TLS      =", EMAIL_USE_TLS)
print("EMAIL_USE_SSL      =", EMAIL_USE_SSL)
print("EMAIL_TIMEOUT      =", EMAIL_TIMEOUT)
print("DEFAULT_FROM_EMAIL =", DEFAULT_FROM_EMAIL)
print("FROM               =", FROM)
print("TEST_TO            =", TEST_TO)

In [ ]:
# 4) Basic sanity checks
problems = []
if not EMAIL_HOST:
    problems.append("EMAIL_HOST is missing.")
if not TEST_TO or "@" not in TEST_TO:
    problems.append("TEST_TO is not a valid e-mail address.")
if problems:
    raise SystemExit("Config problems: " + "; ".join(problems))

# 5) DNS resolution
resolve_host(EMAIL_HOST)

# 6) Probes
if EMAIL_USE_TLS and not EMAIL_USE_SSL:
    try_starttls(EMAIL_HOST, EMAIL_PORT, EMAIL_HOST_USER, EMAIL_HOST_PASSWORD, FROM, TEST_TO, timeout=EMAIL_TIMEOUT, perform_send=SEND_EMAIL)

# Always try SSL(465) as a secondary path
try_ssl(EMAIL_HOST, 465, EMAIL_HOST_USER, EMAIL_HOST_PASSWORD, FROM, TEST_TO, timeout=EMAIL_TIMEOUT, perform_send=SEND_EMAIL)

## Optional: Django `send_mail`

This cell uses Django's configured e-mail backend. If your `EMAIL_BACKEND` is the console backend in development, the message will print to the console rather than go out over SMTP.

In [ ]:
try:
    from django.core.mail import send_mail
    from django.conf import settings as djset
    print("Using Django backend:", getattr(djset, "EMAIL_BACKEND", None))
    if SEND_EMAIL:
        n = send_mail(
            subject="Django e-mail probe (ENV-first)",
            message="Hello from OurFinanceTracker (ENV-first probe).",
            from_email=FROM,
            recipient_list=[TEST_TO],
            fail_silently=False,
        )
        print("send_mail returned:", n)
    else:
        print("SEND_EMAIL=False; skipping Django send_mail.")
except Exception as e:
    print("Django send_mail failed:", e)
    import traceback; traceback.print_exc()

### Reading the output

- If **STARTTLS 587** works and sends, prefer that in production.
- If **465 SSL** fails with a timeout, keep using 587; verify firewall and mail server settings if you truly need 465.
- If you see **DNS resolution issues**, ensure `EMAIL_HOST` is a **DNS-only** record (not proxied) pointing to your mail server (e.g., `mail.yourdomain.com`).

This notebook does **not** override your environment; it reads from it and only falls back to Django settings if an environment variable is missing.